### Векторизация всех документов

In [ ]:
# импорт библиотек
import json
import uuid
import torch

from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams

In [12]:
# загрузка заранее разбитых документов
input_file_path = "./processed_output/all_documents.json"

# Чтение данных из JSON
with open(input_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Восстановление массива объектов Document
documents = [
    Document(metadata=item["metadata"], page_content=item["page_content"])
    for item in data
]

print(f"Загружено {len(documents)} документов.")

Загружено 2328 документов.


In [43]:
# Устройство по умолчанию для тензоров
default_device = torch.device("cpu")  # CPU используется по умолчанию
print(f"Устройство по умолчанию: {default_device}")

# Проверить, где находится модель
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)
print(f"Устройство модели: {next(model.parameters()).device}")

Устройство по умолчанию: cpu
Устройство модели: mps:0


In [44]:
# Использование GPU через MPS (для macOS)
if torch.backends.mps.is_available():
    model.to("mps")
    print("Модель переведена на MPS.")
else:
    print("MPS недоступен, используется CPU.")

Модель переведена на MPS.


In [21]:
documents[1599]

Document(metadata={'title': '6.8 Общая характеристика качества воды реки Кубани и её притоков', 'level': 2, 'parent_title': '6 Гидрологическая характеристика речного бассейна', 'start_page': 171, 'end_page': 172, 'file': 'Книга_1.json', 'basin': 'Кубань', 'chunk_index': 2, 'total_chunks': 2, 'order': 1600}, page_content='. Содержание ионов аммония и нитритов в воде р. Кубани и ее притоков не превышало Пдк ни в одном створе. Содержание ионов нитритов не превышало Пдк в верховьях реки до створа ниже г. Черкесска. В створе ниже г.Черкесска содержание азота нитритного составляло 1,9 Пдк и было максимальным по стволу р.Кубани. От ст. Беломечетской до г. Невинномысска концентрация азота нитритного в воде уменьшалась до значений ниже ПДКр. и была стабильна до г. Краснодара. От створа г.Краснодар, Тургеневский мост, где концентрация его увеличилась, содержание в воде составило 1,1 ПДКр. Азот нитриный присутствовал в каждом створе до Тиховского Г У, далее по руслу р.Кубани содержание снижалось 

In [57]:
%%time
model.to("mps")
query = "О чем говорится в Таблица 5.63? "
query_vector = model.encode(query).tolist()

CPU times: user 33.8 ms, sys: 19.8 ms, total: 53.5 ms
Wall time: 46.1 ms


In [59]:
%%time
model.to('cpu')
query = "О чем говорится в Таблица 5.63? "
query_vector = model.encode(query).tolist()

CPU times: user 112 ms, sys: 95.9 ms, total: 208 ms
Wall time: 186 ms


### Векторизация 

In [41]:
# Модель для векторизации
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)

model.to('mps')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [42]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [26]:
# Подключение к удалённому серверу Qdrant
client = QdrantClient(url="http://192.168.137.253:6333")

# Модель для векторизации
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)

# Имя коллекции
collection_name = "documents_collection"

# Проверка или создание коллекции
collections = client.get_collections().collections
if not any(c.name == collection_name for c in collections):
    vector_size = model.get_sentence_embedding_dimension()
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance="Cosine"),
    )
    print(f"Коллекция '{collection_name}' создана.")
else:
    print(f"Коллекция '{collection_name}' уже существует.")

Коллекция 'documents_collection' уже существует.


In [24]:
documents_new = documents[1600:]

In [27]:
# Размер батча
batch_size = 100

# Загрузка данных в базу по мере обработки
batch_points = []
for idx, doc in enumerate(documents_new, start=1):
    # Генерация вектора
    vector = model.encode(doc.page_content).tolist()

    # Создание структуры точки
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload={"content": doc.page_content, **doc.metadata},
    )
    batch_points.append(point)

    # Если накоплено достаточно точек, отправляем их в базу
    if len(batch_points) >= batch_size:
        client.upsert(collection_name=collection_name, points=batch_points)
        print(f"Добавлено {len(batch_points)} точек в коллекцию '{collection_name}'.")
        batch_points = []  # Очищаем батч

# Загрузка оставшихся точек
if batch_points:
    client.upsert(collection_name=collection_name, points=batch_points)
    print(f"Добавлено {len(batch_points)} точек в коллекцию '{collection_name}'.")

print("Все данные успешно загружены.")

Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 100 точек в коллекцию 'documents_collection'.
Добавлено 28 точек в коллекцию 'documents_collection'.
Все данные успешно загружены.


In [3]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# Подключение к удалённому серверу Qdrant
client = QdrantClient(url="http://192.168.137.253:6333")

# Модель для векторизации
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)

query = "О чем говорится в Таблица 5.63? "
query_vector = model.encode(query).tolist()

# Поиск в каждой коллекции
search_results = client.search(
        collection_name='documents_collection',
        query_vector=query_vector,
        limit=5,
)
for result in search_results:
    print(f"Score: {result.score}")
    print(f"Content: {result.payload['content']}")
    print(f"ID: {result.id}")
    print("-" * 40)

/Users/anastasia/docs/Projects/skiovo_project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Score: 0.6145767
Content: Продолжение таблицы 6.3
ID: ce20db24-f76a-4a21-a446-9b3d70e34c8a
----------------------------------------
Score: 0.5854373
Content: Продолжение таблицы 5.25
ID: b26239eb-d2b9-4c6b-a904-60ebd3e5a53e
----------------------------------------
Score: 0.5854373
Content: Продолжение таблицы 5.25
ID: 62de426f-5c7b-41c5-b6ea-738b7d7df43c
----------------------------------------
Score: 0.54746294
Content: Продолжение таблицы 8.3
ID: 8c013ff2-8ec9-4e73-9aac-3159448e52a9
----------------------------------------
Score: 0.4918947
Content: Таблица 5.3   Участок II  г.Лиски – г.Павловск, млн.м3
ID: 9cfd5a67-8886-400e-893f-114e938c4ec9
----------------------------------------


In [38]:
# Ваш запрос
query = "О чем говорится в Таблица 5.63? "
query_vector = model.encode(query).tolist()

# Поиск в коллекции
search_results = client.search(
    collection_name='documents_collection',
    query_vector=query_vector,
    limit=5,
)

# Обработка результатов
for result in search_results:
    print(f"Score: {result.score}")
    print(f"Content: {result.payload['content']}")
    print(f"ID: {result.id}")

    # Извлечение вектора для данной точки
    retrieved_point = client.retrieve(
        collection_name='documents_collection',
        ids=[result.id],  # ID точки
        with_vectors=True,  # Включаем вектор
        with_payload=False  # Метаданные уже есть
    )
    
    # Проверка и вывод вектора
    if retrieved_point:
        vector = retrieved_point[0].vector
        print(f"Vector: {vector[:10]}...")  # Вывод первых 10 элементов вектора для наглядности
        print(f'длина: {len(vector)}')
    else:
        print("Vector not found.")
    print("-" * 40)

Score: 0.61457676
Content: Продолжение таблицы 6.3
ID: ce20db24-f76a-4a21-a446-9b3d70e34c8a
Vector: [0.032952294, 0.020080041, -0.0089206435, 0.01283157, -0.0017048516, 0.0034097582, 0.036392625, -0.022825463, 0.003623623, -0.0049222913]...
длина: 1024
----------------------------------------
Score: 0.58543724
Content: Продолжение таблицы 5.25
ID: b26239eb-d2b9-4c6b-a904-60ebd3e5a53e
Vector: [0.0055511813, 0.0067154635, -0.019577155, 0.050584856, -0.007987044, 0.0055501712, 0.0013262462, 0.016454509, 0.011005821, 0.035359535]...
длина: 1024
----------------------------------------
Score: 0.58543724
Content: Продолжение таблицы 5.25
ID: 62de426f-5c7b-41c5-b6ea-738b7d7df43c
Vector: [0.0055511813, 0.0067154635, -0.019577155, 0.050584856, -0.007987044, 0.0055501712, 0.0013262462, 0.016454509, 0.011005821, 0.035359535]...
длина: 1024
----------------------------------------
Score: 0.5474629
Content: Продолжение таблицы 8.3
ID: 8c013ff2-8ec9-4e73-9aac-3159448e52a9
Vector: [-0.05333955, -0.00